## Preperation: OpenAI API KEY

In [7]:
import json

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv("OPENAI_API_KEY"))  # should show your key


## Agent1: Domain Detector

In [3]:
!pip install -U langchain-core langchain-openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
!pip install -U langgraph

In [ ]:
!pip install langchain-openai langchain-core langgraph

### TBF-Test 0 for Agent_D: with a fixed knowledge database

In [14]:
# ---------- imports ----------
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langgraph.graph import StateGraph, END

import json, re
from typing import List, Dict, Any, Optional

# ---------- state ----------
class DomainDetectorState(dict):
    def __init__(
        self,
        columns: List[str],
        column_types: Dict[str, str],
        domains_found: List[Dict[str, Any]] = None,
        current_domain: Optional[str] = None,
        jargon_terms: List[str] = None,
        remaining_columns: List[str] = None,
        iteration: int = 0,
        messages: List[str] = None,
    ):
        super().__init__(
            columns=columns,
            column_types=column_types,
            domains_found=domains_found or [],
            current_domain=current_domain,
            jargon_terms=jargon_terms or [],
            remaining_columns=remaining_columns or list(columns),
            iteration=iteration,
            messages=messages or [],
        )

# ---------- LLM ----------
llm = OpenAI(temperature=0)
# llm = OpenAI(model_name="gpt-4o-mini", temperature=0)

# ---------- think ----------
def think(state: DomainDetectorState) -> DomainDetectorState:
    if len(state["domains_found"]) >= 5 or not state["remaining_columns"]:
        return state

    prompt = PromptTemplate.from_template(
        """
        You are a domain expert.
        Remaining columns: {remaining_columns}
        Column types: {column_types}
        Domains already found: {domains_found}
        Identify ONE new domain (single word / short phrase).
        """
    )
    name = LLMChain(llm=llm, prompt=prompt).run(
        remaining_columns=state["remaining_columns"],
        column_types=state["column_types"],
        domains_found=[d["domain"] for d in state["domains_found"]],
    ).strip()

    new = state.copy()
    new["current_domain"] = name
    new["messages"].append(f"Think → {name}")
    return new

# ---------- helper ----------
def search_examples(domain: str, n: int = 5) -> List[str]:
    raw = (prompt | llm | StrOutputParser()).invoke({"domain": domain, "n": n})

    # ① Strict JSON path
    try:
        terms = json.loads(raw)
        if isinstance(terms, list) and len(terms) >= 3:
            return terms[:n]
    except Exception:
        pass
    
    match = re.search(r"\[(.*?)\]", raw_response, re.S)
    if match:
        rough = re.split(r"[\"',\[\]]+", match.group(1))
        cleaned = [t.strip() for t in rough if t.strip()]
        if cleaned:
            return cleaned[:n_terms]

    # 5️⃣  Final generic fallback
    return ["metric", "indicator", "analysis", "benchmark", "trend"][:n_terms]

# ---------- act ----------
def act(state: DomainDetectorState) -> DomainDetectorState:
    if not state["current_domain"]:
        new = state.copy(); new["iteration"] += 1; return new

    examples = search_examples(state["current_domain"])

    prompt = PromptTemplate.from_template(
        """
        Domain: {domain}
        Columns: {columns}
        Example jargon: {examples}
        Produce ≥5 NEW jargon terms (JSON array).
        """
    )
    jargon_response = LLMChain(llm=llm, prompt=prompt).run(
        domain=state["current_domain"],
        columns=state["columns"],
        examples=examples,
    )
    try:
        array_txt = re.search(r\"\\[.*?\\]\", jargon_response, re.S).group(0)
        jargon_terms = json.loads(array_txt)
    except Exception:
        jargon_terms = []

    new = state.copy()
    new[\"jargon_terms\"] = jargon_terms        # <- store the *parsed* list
    new[\"messages\"].append(f\"Act → {jargon_terms}\")
    new[\"iteration\"] += 1
    return new

# ---------- reflect ----------
def reflect(state: DomainDetectorState) -> DomainDetectorState:
    if not state["current_domain"] or not state["jargon_terms"]:
        new_state = state.copy()
        new_state[\"iteration\"] += 1
        return new_state

    cols_lower = {c.lower() for c in state["columns"]}
    valid = [t for t in state["jargon_terms"] if t.lower() not in cols_lower]

    new = state.copy()
    if len(valid) >= 3:
        new["domains_found"].append(
            {"domain": state["current_domain"], "jargon_terms": valid[:5]}
        )
        new["messages"].append(f"Reflect ✔ {state['current_domain']}")
        if new["remaining_columns"]:
            new["remaining_columns"].pop(0)
    else:
        new["messages"].append(f"Reflect ✖ {state['current_domain']}")

    new["current_domain"] = None
    new["jargon_terms"] = []
    new["iteration"] += 1
    return new

# ---------- stop rule ----------
def should_end(state: DomainDetectorState) -> str:
    if len(state["domains_found"]) >= 5 or not state["remaining_columns"] or state["iteration"] >= 10:
        return "end"
    return "continue"

# ---------- graph ----------
def build_graph():
    g = StateGraph(DomainDetectorState)
    g.add_node("think", think)
    g.add_node("act", act)
    g.add_node("reflect", reflect)
    g.add_edge("think", "act")
    g.add_edge("act", "reflect")
    g.add_conditional_edges("reflect", should_end, {"continue": "think", "end": END})
    g.set_entry_point("think")
    return g.compile()

# ---------- run ----------
def run_domain_detector(columns, column_types=None):
    column_types = column_types or {c: "text" for c in columns}
    state0 = DomainDetectorState(columns=columns, column_types=column_types)
    return build_graph().invoke(state0, config={"recursion_limit": 50})["domains_found"]

# ---- quick test ----
if __name__ == "__main__":
    cols = ["Quarter", "number_Customers", "Total_Transactions", "Revenue", "Profit"]
    print(json.dumps(run_domain_detector(cols), indent=2))


SyntaxError: unexpected character after line continuation character (1981263389.py, line 77)

In [15]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
import re
from typing import List, Dict, Any, Optional
from langgraph.graph import StateGraph, END

# Define the state schema
class DomainDetectorState(dict):
    """State for the domain detector agent."""
    def __init__(self, 
                 columns: List[str],
                 column_types: Dict[str, str],
                 domains_found: List[Dict[str, Any]] = None,
                 current_domain: Optional[str] = None,
                 jargon_terms: List[str] = None,
                 remaining_columns: List[str] = None,
                 iteration: int = 0,
                 messages: List[str] = None):
        
        self.columns = columns
        self.column_types = column_types
        self.domains_found = domains_found or []
        self.current_domain = current_domain
        self.jargon_terms = jargon_terms or []
        self.remaining_columns = remaining_columns or list(columns)
        self.iteration = iteration
        self.messages = messages or []
        
        super().__init__(
            columns=self.columns,
            column_types=self.column_types,
            domains_found=self.domains_found,
            current_domain=self.current_domain,
            jargon_terms=self.jargon_terms,
            remaining_columns=self.remaining_columns,
            iteration=self.iteration,
            messages=self.messages
        )

# Initialize LLM
llm = OpenAI(temperature=0)

# Define the "Think" node - Identify a potential domain
def think(state: DomainDetectorState) -> DomainDetectorState:
    """Identify a potential domain based on remaining columns."""
    
    # Skip if we already have 5 domains or no columns remain
    if len(state["domains_found"]) >= 5 or not state["remaining_columns"]:
        return state
    
    think_template = """
    You are a domain expert tasked with identifying knowledge domains from a set of data columns.
    
    Current columns to analyze: {remaining_columns}
    Column types: {column_types}
    
    Domains already identified: {domains_found}
    
    Based on the remaining columns, identify ONE new knowledge domain that best represents them.
    Consider business, scientific, or technical domains that would use such terminology.
    
    Return only the domain name as a single word or short phrase.
    """
    
    think_prompt = PromptTemplate(
        template=think_template,
        input_variables=["remaining_columns", "column_types", "domains_found"]
    )
    
    think_chain = LLMChain(llm=llm, prompt=think_prompt)
    
    domain = think_chain.run({
        "remaining_columns": state["remaining_columns"],
        "column_types": state["column_types"],
        "domains_found": [d["domain"] for d in state["domains_found"]]
    }).strip()
    
    new_state = state.copy()
    new_state["current_domain"] = domain
    new_state["messages"].append(f"Identified potential domain: {domain}")
    
    return new_state

# Mock function for search_examples
def search_examples(domain: str) -> List[str]:
    """Mock function to simulate fetching jargon hints for a domain."""
    # In a real implementation, this would call an external API or database
    domain_examples = {
        "Finance": ["ROI", "EBITDA", "liquidity", "amortization", "depreciation"],
        "Retail": ["SKU", "inventory turnover", "markdown", "POS", "shrinkage"],
        "E-commerce": ["conversion rate", "cart abandonment", "AOV", "CPC", "CTR"],
        "Sales": ["pipeline", "lead generation", "churn rate", "upselling", "quota"],
        "Marketing": ["CAC", "LTV", "engagement rate", "attribution", "funnel"],
        "Business Intelligence": ["KPI", "dashboard", "data warehouse", "ETL", "OLAP"]
    }
    
    # Default examples for domains not in our mock database
    default_examples = ["metric", "indicator", "analysis", "benchmark", "trend"]
    
    return domain_examples.get(domain, default_examples)

# Define the "Act" node - Get jargon terms
def act(state: DomainDetectorState) -> DomainDetectorState:
    
    """Generate jargon terms for the current domain."""

    # Skip only if there is *no* domain to work on
    if not state["current_domain"]:
        new_state = state.copy()
        new_state["iteration"] += 1
        return new_state
    
    # Get example jargon terms
    example_terms = search_examples(state["current_domain"])
    
    act_template = """
    You are a domain expert in {domain}.
    
    Your task is to generate at least 5 jargon terms that are commonly used in the {domain} domain 
    but do NOT appear verbatim in the following column names: {columns}.
    
    Here are some example jargon terms for this domain: {examples}
    
    Return a JSON array of strings containing ONLY the jargon terms.
    Example: ["term1", "term2", "term3", "term4", "term5"]
    """
    
    act_prompt = PromptTemplate(
        template=act_template,
        input_variables=["domain", "columns", "examples"]
    )
    
    act_chain = LLMChain(llm=llm, prompt=act_prompt)
    
    jargon_response = act_chain.run({
        "domain": state["current_domain"],
        "columns": state["columns"],
        "examples": example_terms
    })
    
    # Extract the JSON array from the response
    try:
        # Find anything that looks like a JSON array
        match = re.search(r'\[.*\]', jargon_response, re.DOTALL)
        if match:
            jargon_terms = json.loads(match.group(0))
        else:
            jargon_terms = []
    except Exception:
        jargon_terms = []
    
    new_state = state.copy()
    new_state["jargon_terms"] = jargon_terms
    new_state["messages"].append(f"Generated jargon terms: {jargon_terms}")
    new_state["iteration"] += 1
    return new_state

# Define the "Reflect" node - Validate jargon terms
def reflect(state: DomainDetectorState) -> DomainDetectorState:
    """Validate jargon terms and update domains found."""
    
    if not state["current_domain"] or not state["jargon_terms"]:
        return state
    
    # Convert column names to lowercase for case-insensitive comparison
    columns_lower = [col.lower() for col in state["columns"]]
    
    # Filter out jargon terms that appear in column names
    valid_terms = []
    for term in state["jargon_terms"]:
        if term.lower() not in columns_lower:
            valid_terms.append(term)
    
    # If we have at least 3 valid terms, add the domain to our list
    new_state = state.copy()
    if len(valid_terms) >= 3:
        domain_entry = {
            "domain": state["current_domain"],
            "jargon_terms": valid_terms[:5]  # Limit to 5 terms
        }
        new_state["domains_found"].append(domain_entry)
        new_state["messages"].append(f"Added domain: {state['current_domain']} with terms: {valid_terms[:5]}")
        
        # Remove columns that were used for this domain (optional)
        # This is a simplified approach - in a real implementation, you might want
        # to use the LLM to determine which columns were used for this domain
        if len(new_state["remaining_columns"]) > 0:
            new_state["remaining_columns"].pop(0)
    else:
        new_state["messages"].append(f"Rejected domain: {state['current_domain']} - not enough valid jargon terms")
    
    # Reset current domain and jargon terms
    new_state["current_domain"] = None
    new_state["jargon_terms"] = []
    new_state["iteration"] += 1
    
    return new_state

# Define the condition to end the graph
def should_end(state: DomainDetectorState) -> str:
    """Determine if the graph should end."""
    if len(state["domains_found"]) >= 5:
        return "end"
    if not state["remaining_columns"]:
        return "end"
    if state["iteration"] >= 10:  # Safety limit
        return "end"
    return "continue"

# Build the graph
def build_domain_detector_graph():
    """Build and return the domain detector graph."""
    workflow = StateGraph(DomainDetectorState)
    
    # Add nodes
    workflow.add_node("think", think)
    workflow.add_node("act", act)
    workflow.add_node("reflect", reflect)
    
    # Add edges
    workflow.add_edge("think", "act")
    workflow.add_edge("act", "reflect")
    
    # Add conditional edge
    workflow.add_conditional_edges(
        "reflect",
        should_end,
        {
            "continue": "think",
            "end": END
        }
    )
    
    # Set the entry point
    workflow.set_entry_point("think")
    
    return workflow.compile()

# Function to run the domain detector
def run_domain_detector(columns, column_types=None):
    """
    Run the domain detector on the given columns.
    
    Args:
        columns (list): List of column names
        column_types (dict, optional): Dictionary mapping column names to their data types
    
    Returns:
        list: List of domains with their jargon terms
    """
    if column_types is None:
        column_types = {col: "text" for col in columns}
    
    # Initialize the state
    initial_state = DomainDetectorState(
        columns=columns,
        column_types=column_types
    )
    
    # Build and run the graph
    graph = build_domain_detector_graph()
    final_state = graph.invoke(initial_state)
    
    return final_state["domains_found"]

# Example usage
if __name__ == "__main__":
    columns = ["Quarter", "number_Customers", "Total_Transactions", "Revenue", "Profit"]
    column_types = {
        "Quarter": "object",
        "number_Customers": "int",
        "Total_Transactions": "float",
        "Revenue": "float",
        "Profit": "float"
    }
    
    domains = run_domain_detector(columns, column_types)
    print(json.dumps(domains, indent=2))


[
  {
    "domain": "Business Analytics",
    "jargon_terms": [
      "KPI",
      "ROI",
      "Forecasting",
      "Segmentation",
      "Data Mining"
    ]
  },
  {
    "domain": "Business Finance",
    "jargon_terms": [
      "ROI",
      "EBITDA",
      "Liquidity",
      "Cash Flow",
      "Capital Expenditure"
    ]
  },
  {
    "domain": "Business Economics",
    "jargon_terms": [
      "demand_curve",
      "elasticity",
      "marginal_cost",
      "opportunity_cost",
      "market_share"
    ]
  },
  {
    "domain": "Business Accounting",
    "jargon_terms": [
      "ledger",
      "depreciation",
      "amortization",
      "accrual",
      "cash flow"
    ]
  },
  {
    "domain": "Business Management",
    "jargon_terms": [
      "KPI",
      "ROI",
      "SWOT",
      "P&L",
      "Forecast"
    ]
  }
]


#### - Visualization

In [43]:
!git filter-repo --path .env --invert-paths
!git push origin main --force


git: 'filter-repo' is not a git command. See 'git --help'.
Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 8 threads
Compressing objects: 100% (14/14), done.
Writing objects: 100% (16/16), 14.48 KiB | 7.24 MiB/s, done.
Total 16 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 1 local object.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— OpenAI API

In [44]:
!brew install git-filter-repo


zsh:1: command not found: brew


In [45]:
!pip install git-filter-repo  # May not work on all setups


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [46]:
!git filter-repo --path .env --invert-paths


Aborting: Refusing to destructively overwrite repo history since
this does not look like a fresh clone.
  (expected freshly packed repo)
Please operate on a fresh clone instead.  If you want to proceed
anyway, use --force.


In [47]:
!git push origin main --force


Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 8 threads
Compressing objects: 100% (14/14), done.
Writing objects: 100% (16/16), 14.48 KiB | 7.24 MiB/s, done.
Total 16 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 1 local object.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— OpenAI API Key ————————————————————————————————————
remote:        lo

In [48]:
!git filter-repo --path .env --invert-paths --force


NOTICE: Removing 'origin' remote; see 'Why is my origin removed?'
        in the manual if you want to push back there.
        (was https://github.com/77luvC/D2D_Data2Dashboard.git)
Parsed 10 commits
New history written in 0.06 seconds; now repacking/cleaning...
Repacking your repo and cleaning out old unneeded objects
HEAD is now at 22fed46 agentize domain detector
Enumerating objects: 32, done.
Counting objects: 100% (32/32), done.
Delta compression using up to 8 threads
Compressing objects: 100% (29/29), done.
Writing objects: 100% (32/32), done.
Total 32 (delta 10), reused 1 (delta 0), pack-reused 0
Completely finished after 0.14 seconds.


In [49]:
!git push origin main --force


fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [50]:
!git remote add origin https://github.com/77luvC/D2D_Data2Dashboard.git


In [51]:
!git push origin main --force


Enumerating objects: 32, done.
Counting objects: 100% (32/32), done.
Delta compression using up to 8 threads
Compressing objects: 100% (19/19), done.
Writing objects: 100% (32/32), 39.08 KiB | 39.08 MiB/s, done.
Total 32 (delta 10), reused 32 (delta 10), pack-reused 0
remote: Resolving deltas: 100% (10/10), done.
To https://github.com/77luvC/D2D_Data2Dashboard.git
 + e96d856...22fed46 main -> main (forced update)


### Test 1 for Agent_D: Dynamic LLM-powered jargon fetcher

In [17]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import json
import re
from typing import List, Dict, Any, Optional
from langgraph.graph import StateGraph, END

# Define the state schema
class DomainDetectorState(dict):
    """State for the domain detector agent."""
    def __init__(self, 
                 columns: List[str],
                 column_types: Dict[str, str],
                 domains_found: List[Dict[str, Any]] = None,
                 current_domain: Optional[str] = None,
                 jargon_terms: List[str] = None,
                 remaining_columns: List[str] = None,
                 iteration: int = 0,
                 messages: List[str] = None):
        
        self.columns = columns
        self.column_types = column_types
        self.domains_found = domains_found or []
        self.current_domain = current_domain
        self.jargon_terms = jargon_terms or []
        self.remaining_columns = remaining_columns or list(columns)
        self.iteration = iteration
        self.messages = messages or []
        
        super().__init__(
            columns=self.columns,
            column_types=self.column_types,
            domains_found=self.domains_found,
            current_domain=self.current_domain,
            jargon_terms=self.jargon_terms,
            remaining_columns=self.remaining_columns,
            iteration=self.iteration,
            messages=self.messages
        )

# Initialize LLM
llm = OpenAI(temperature=0)

# Define the "Think" node - Identify a potential domain
def think(state: DomainDetectorState) -> DomainDetectorState:
    """Identify a potential domain based on remaining columns."""
    
    # Skip if we already have 5 domains or no columns remain
    if len(state["domains_found"]) >= 5 or not state["remaining_columns"]:
        return state
    
    think_template = """
    You are a domain expert tasked with identifying knowledge domains from a set of data columns.
    
    Current columns to analyze: {remaining_columns}
    Column types: {column_types}
    
    Domains already identified: {domains_found}
    
    Based on the remaining columns, identify ONE new knowledge domain that best represents them.
    Consider business, scientific, or technical domains that would use such terminology.
    
    Return only the domain name as a single word or short phrase.
    """
    
    think_prompt = PromptTemplate(
        template=think_template,
        input_variables=["remaining_columns", "column_types", "domains_found"]
    )
    
    think_chain = LLMChain(llm=llm, prompt=think_prompt)
    
    domain = think_chain.run({
        "remaining_columns": state["remaining_columns"],
        "column_types": state["column_types"],
        "domains_found": [d["domain"] for d in state["domains_found"]]
    }).strip()
    
    new_state = state.copy()
    new_state["current_domain"] = domain
    new_state["messages"].append(f"Identified potential domain: {domain}")
    
    return new_state

# -------------------------------------------------------------------
# Dynamic LLM-powered jargon fetcher
# -------------------------------------------------------------------
def search_examples(domain: str, n_terms: int = 5, llm_model=llm) -> List[str]:
    """
    Query the LLM for ~n_terms canonical jargon / acronyms that typify `domain`.
    Falls back to a small generic list if the model response is unusable.
    """

    # 1️⃣  Compose the prompt
    prompt = PromptTemplate.from_template(
        """
        You are a senior specialist in {domain}.
        List {n} widely-used jargon terms or acronyms **unique** to this field.
        Return ONLY a valid JSON array of quoted strings, nothing else.
        Example → ["term1","term2",...]
        """
    )

    # 2️⃣  Run the model
    raw_response: str = (
        prompt
        | llm_model
        | StrOutputParser()         # gives us the raw string
    ).invoke({"domain": domain, "n": n_terms})

    # 3️⃣  Try strict JSON parsing first
    try:
        terms = json.loads(raw_response)
        if isinstance(terms, list) and len(terms) >= 3:
            return terms[:n_terms]
    except Exception:
        pass

    # 4️⃣  Regex rescue for “almost JSON”
    match = re.search(r"\[(.*?)\]", raw_response, re.S)
    if match:
        rough = re.split(r"[\"',\[\]]+", match.group(1))
        cleaned = [t.strip() for t in rough if t.strip()]
        if cleaned:
            return cleaned[:n_terms]

    # 5️⃣  Final generic fallback
    return ["metric", "indicator", "analysis", "benchmark", "trend"][:n_terms]


# Define the "Act" node - Get jargon terms
def act(state: DomainDetectorState) -> DomainDetectorState:
    
    """Generate jargon terms for the current domain."""

    # Skip only if there is *no* domain to work on
    if not state["current_domain"]:
        new_state = state.copy()
        new_state["iteration"] += 1
        return new_state
    
    # Get example jargon terms
    example_terms = search_examples(state["current_domain"])
    
    act_template = """
    You are a domain expert in {domain}.
    
    Your task is to generate at least 5 jargon terms that are commonly used in the {domain} domain 
    but do NOT appear verbatim in the following column names: {columns}.
    
    Here are some example jargon terms for this domain: {examples}
    
    Return a JSON array of strings containing ONLY the jargon terms.
    Example: ["term1", "term2", "term3", "term4", "term5"]
    """
    
    act_prompt = PromptTemplate(
        template=act_template,
        input_variables=["domain", "columns", "examples"]
    )
    
    act_chain = LLMChain(llm=llm, prompt=act_prompt)
    
    jargon_response = act_chain.run({
        "domain": state["current_domain"],
        "columns": state["columns"],
        "examples": example_terms
    })
    
    # Extract the JSON array from the response
    try:
        # Find anything that looks like a JSON array
        match = re.search(r'\[.*\]', jargon_response, re.DOTALL)
        if match:
            jargon_terms = json.loads(match.group(0))
        else:
            jargon_terms = []
    except Exception:
        jargon_terms = []
    
    new_state = state.copy()
    new_state["jargon_terms"] = jargon_terms
    new_state["messages"].append(f"Generated jargon terms: {jargon_terms}")
    new_state["iteration"] += 1
    return new_state

# Define the "Reflect" node - Validate jargon terms
def reflect(state: DomainDetectorState) -> DomainDetectorState:
    """Validate jargon terms and update domains found."""
    
    if not state["current_domain"] or not state["jargon_terms"]:
        return state
    
    # Convert column names to lowercase for case-insensitive comparison
    columns_lower = [col.lower() for col in state["columns"]]
    
    # Filter out jargon terms that appear in column names
    valid_terms = []
    for term in state["jargon_terms"]:
        if term.lower() not in columns_lower:
            valid_terms.append(term)
    
    # If we have at least 3 valid terms, add the domain to our list
    new_state = state.copy()
    if len(valid_terms) >= 3:
        domain_entry = {
            "domain": state["current_domain"],
            "jargon_terms": valid_terms[:5]  # Limit to 5 terms
        }
        new_state["domains_found"].append(domain_entry)
        new_state["messages"].append(f"Added domain: {state['current_domain']} with terms: {valid_terms[:5]}")
        
        # Remove columns that were used for this domain (optional)
        # This is a simplified approach - in a real implementation, you might want
        # to use the LLM to determine which columns were used for this domain
        if len(new_state["remaining_columns"]) > 0:
            new_state["remaining_columns"].pop(0)
    else:
        new_state["messages"].append(f"Rejected domain: {state['current_domain']} - not enough valid jargon terms")
    
    # Reset current domain and jargon terms
    new_state["current_domain"] = None
    new_state["jargon_terms"] = []
    new_state["iteration"] += 1
    
    return new_state

# Define the condition to end the graph
def should_end(state: DomainDetectorState) -> str:
    """Determine if the graph should end."""
    if len(state["domains_found"]) >= 5:
        return "end"
    if not state["remaining_columns"]:
        return "end"
    if state["iteration"] >= 10:  # Safety limit
        return "end"
    return "continue"

# Build the graph
def build_domain_detector_graph():
    """Build and return the domain detector graph."""
    workflow = StateGraph(DomainDetectorState)
    
    # Add nodes
    workflow.add_node("think", think)
    workflow.add_node("act", act)
    workflow.add_node("reflect", reflect)
    
    # Add edges
    workflow.add_edge("think", "act")
    workflow.add_edge("act", "reflect")
    
    # Add conditional edge
    workflow.add_conditional_edges(
        "reflect",
        should_end,
        {
            "continue": "think",
            "end": END
        }
    )
    
    # Set the entry point
    workflow.set_entry_point("think")
    
    return workflow.compile()

# Function to run the domain detector
def run_domain_detector_test1(columns, column_types=None):
    """
    Run the domain detector on the given columns.
    
    Args:
        columns (list): List of column names
        column_types (dict, optional): Dictionary mapping column names to their data types
    
    Returns:
        list: List of domains with their jargon terms
    """
    if column_types is None:
        column_types = {col: "text" for col in columns}
    
    # Initialize the state
    initial_state = DomainDetectorState(
        columns=columns,
        column_types=column_types
    )
    
    # Build and run the graph
    graph = build_domain_detector_graph()
    final_state = graph.invoke(initial_state)
    
    return final_state["domains_found"]

# Example usage
if __name__ == "__main__":
    columns = ["Quarter", "number_Customers", "Total_Transactions", "Revenue", "Profit"]
    column_types = {
        "Quarter": "object",
        "number_Customers": "int",
        "Total_Transactions": "float",
        "Revenue": "float",
        "Profit": "float"
    }
    
    domains = run_domain_detector_test1(columns, column_types)
    print(json.dumps(domains, indent=2))


[
  {
    "domain": "Business Analytics",
    "jargon_terms": [
      "Data Mining",
      "Predictive Modeling",
      "Data Visualization",
      "Descriptive Analytics",
      "Prescriptive Analytics"
    ]
  },
  {
    "domain": "Business Finance",
    "jargon_terms": [
      "EBITDA",
      "ROI",
      "DCF",
      "IRR",
      "NPV"
    ]
  },
  {
    "domain": "Business Economics",
    "jargon_terms": [
      "Demand Curve",
      "Opportunity Cost",
      "Elasticity",
      "Monopoly",
      "Oligopoly"
    ]
  },
  {
    "domain": "Business Accounting",
    "jargon_terms": [
      "Balance Sheet",
      "Cash Flow Statement",
      "Accrual Basis",
      "Depreciation",
      "Inventory Valuation"
    ]
  },
  {
    "domain": "Business Management",
    "jargon_terms": [
      "KPI",
      "ROI",
      "SWOT",
      "B2B",
      "B2C"
    ]
  }
]


### 1. Cleaned Domain-Concept Pairs

In [57]:
from collections import Counter

# keep the first occurrence of any jargon term;
# drop later duplicates that show up in other domains.

def get_first_unique_pairs(domains_found):
    """
    Keep the first‑seen (domain, jargon) pair for every jargon_term.
    Later occurrences of the same jargon_term in other domains are skipped.
    """
    seen = set()
    unique_pairs = []

    for entry in domains_found:          # keep original order
        dom = entry["domain"]
        for term in entry["jargon_terms"]:
            if term not in seen:         # only accept the first time we see it
                unique_pairs.append({"domain": dom, "jargon_terms": term})
                seen.add(term)

    return unique_pairs

cleaned = get_first_unique_pairs(domains)
import json, pprint; pprint.pp(cleaned, width=60)



[{'domain': 'Business Analytics',
  'jargon_terms': 'Data Mining'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Predictive Modeling'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Data Visualization'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Descriptive Analytics'},
 {'domain': 'Business Analytics',
  'jargon_terms': 'Prescriptive Analytics'},
 {'domain': 'Business Finance', 'jargon_terms': 'EBITDA'},
 {'domain': 'Business Finance', 'jargon_terms': 'ROI'},
 {'domain': 'Business Finance', 'jargon_terms': 'DCF'},
 {'domain': 'Business Finance', 'jargon_terms': 'IRR'},
 {'domain': 'Business Finance', 'jargon_terms': 'NPV'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Demand Curve'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Opportunity Cost'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Elasticity'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Monopoly'},
 {'domain': 'Business Economics',
  'jargon_terms': 'Oligopoly'},
 

### 2. Filter by yes/no

In [77]:
# ------------ 1. imports & chat object ------------
from langchain_openai import ChatOpenAI
from typing import List, Dict, Union

# feel free to tweak these
MODEL_NAME   = "gpt-4o"   # or "gpt-4o", "gpt-3.5-turbo"
TEMPERATURE  = 0
MAX_TOKENS   = 6               # we only need YES / NO
API_KEY_ENV  = "OPENAI_API_KEY"

# singleton chat object (re‑used for every call)
chat = ChatOpenAI(
    model_name   = MODEL_NAME,
    temperature  = TEMPERATURE,
    max_tokens   = MAX_TOKENS,
    openai_api_key = None,     # uses env var by default
)


# ------------ 2. prompt builder ------------
def _build_prompt(jargon:str, domain:str,
                  columns:List[str], column_types:Dict[str,str]) -> str:
    few_shots = """
### Example 1
Data columns: ["Date","Open","High","Low","Close"]
Column types: {"Date":"object","Open":"float","High":"float","Low":"float","Close":"float"}
Jargon term: "EBITDA"
Answer: YES, YES

### Example 2
Data columns: ["User_ID","Session_Duration","Clicks"]
Column types: {"User_ID":"int","Session_Duration":"float","Clicks":"int"}
Jargon term: "Exploratory Data Analysis"
Answer: YES, NO

### Example 3
Data columns: ["Temperature","Humidity"]
Column types: {"Temperature":"float","Humidity":"float"}
Jargon term: "Sharpe Ratio"
Answer: NO, YES
""".strip()

    rule = (
        "First flag = YES if the term is related to the given columns; else NO.\n"
        "Second flag = YES only if the term is normally written as a numeric or algebraic "
        "equation (e.g. ROI = Profit/Cost); else NO.\n"
        "Answer format: YES/NO,YES/NO ‑ no extra words."
    )

    task = f"""
### Task
Domain      : {domain}
Data columns: {json.dumps(columns)}
Column types: {json.dumps(column_types)}
Jargon term : "{jargon}"
{rule}
Answer:"""

    return f"{few_shots}\n{task}"

# ------------- 3. evaluator -------------
def lc_relevance(
        pairs: List[Dict[str, str]],
        columns: List[str],
        column_types: Dict[str, str],
        *,
        keep_only_formula: bool = True,   # <-- new switch
) -> List[Dict[str, Union[str, bool]]]:
    """
    Evaluate every jargon term and (optionally) RETURN ONLY THOSE
    that are both   relevant == True   AND   has_formula == True.

    Set `keep_only_formula=False` if you still want the full list.
    """
    good_rows: list[dict] = []
    all_rows : list[dict] = []

    for p in pairs:
        term   = p.get("jargon_terms") or p.get("concept")
        domain = p["domain"]

        prompt = _build_prompt(term, domain, columns, column_types)
        reply  = chat.invoke(prompt).content.strip().upper()

        # Accept variants such as "YES , NO"
        parts       = [x.strip() for x in reply.split(",")]
        rel_flag    = parts[0].startswith("Y") if parts else False
        formula_flag = parts[1].startswith("Y") if len(parts) > 1 else False

        row = {
            "domain"      : domain,
            "jargon_term" : term,
            "relevant"    : rel_flag,
            "has_formula" : formula_flag,
        }

        all_rows.append(row)
        if rel_flag and formula_flag:
            good_rows.append(row)

    return good_rows if keep_only_formula else all_rows


In [82]:
import pandas as pd

# --- Evaluate every jargon term ------------------------------

evaluated_all = lc_relevance(
        cleaned,
        columns,
        column_types,
        keep_only_formula=False   # return *all* rows so we can inspect the two flags
)

# --- Put results in a nice, readable table -------------------
import pandas as pd
from IPython.display import display  # Jupyter‑friendly

# convert list[dict] → DataFrame
results_df = pd.DataFrame(evaluated_all)

# reorder columns for clarity (optional)
col_order = [
    "domain",
    "jargon_term",
    "relevant",
    "has_formula",
]
results_df = results_df[col_order]

# show the DataFrame
print("\n===== evaluator output (full list) =====\n")
print(results_df.to_string(index=False))  # nice console print




===== evaluator output (full list) =====

             domain            jargon_term  relevant  has_formula
 Business Analytics            Data Mining      True        False
 Business Analytics    Predictive Modeling      True        False
 Business Analytics     Data Visualization      True        False
 Business Analytics  Descriptive Analytics      True        False
 Business Analytics Prescriptive Analytics      True        False
   Business Finance                 EBITDA      True         True
   Business Finance                    ROI      True         True
   Business Finance                    DCF      True         True
   Business Finance                    IRR      True         True
   Business Finance                    NPV      True         True
 Business Economics           Demand Curve      True        False
 Business Economics       Opportunity Cost      True        False
 Business Economics             Elasticity      True         True
 Business Economics              

### 3. Prepare for Formula Retriever

In [89]:
evaluated = lc_relevance(cleaned, columns, column_types)

print(json.dumps(evaluated, indent=2))

[
  {
    "domain": "Business Finance",
    "jargon_term": "EBITDA",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Finance",
    "jargon_term": "ROI",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Finance",
    "jargon_term": "DCF",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Finance",
    "jargon_term": "IRR",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Finance",
    "jargon_term": "NPV",
    "relevant": true,
    "has_formula": true
  },
  {
    "domain": "Business Economics",
    "jargon_term": "Elasticity",
    "relevant": true,
    "has_formula": true
  }
]


In [90]:
print(evaluated)

[{'domain': 'Business Finance', 'jargon_term': 'EBITDA', 'relevant': True, 'has_formula': True}, {'domain': 'Business Finance', 'jargon_term': 'ROI', 'relevant': True, 'has_formula': True}, {'domain': 'Business Finance', 'jargon_term': 'DCF', 'relevant': True, 'has_formula': True}, {'domain': 'Business Finance', 'jargon_term': 'IRR', 'relevant': True, 'has_formula': True}, {'domain': 'Business Finance', 'jargon_term': 'NPV', 'relevant': True, 'has_formula': True}, {'domain': 'Business Economics', 'jargon_term': 'Elasticity', 'relevant': True, 'has_formula': True}]


## Agent2: Formula Fetcher

### 1. Prompt

In [94]:
_GET_FORMULA_TMPL = """\
You are a business‑analytics assistant.

### Task
For the given *jargon term* and *business domain*:

1. State the best‑known **numeric formula** (use valid python‑style identifiers).
2. List every variable that appears in the formula, with a 1‑line definition.

### Format
Return **valid JSON** exactly like:

{{
  "formula"  : "<formula_here>",
  "variables": {{
      "<var1>": "<definition var1>",
      "<var2>": "<definition var2>"
  }}
}}

### Examples
---
Domain     : Business Finance
Jargon term: ROI
JSON:
{{
  "formula"  : "Net_Profit / Investment_Cost",
  "variables": {{
      "Net_Profit"     : "Total profit generated by the investment",
      "Investment_Cost": "Initial cash outlay"
  }}
}}
---
Domain     : Business Finance
Jargon term: EBITDA
JSON:
{{
  "formula"  : "Operating_Income + Depreciation + Amortization",
  "variables": {{
      "Operating_Income": "Earnings from operations before interest & taxes",
      "Depreciation"    : "Expense for tangible asset depreciation",
      "Amortization"    : "Expense for intangible asset amortization"
  }}
}}
---
Now answer for:
Domain     : {domain}
Jargon term: {term}
JSON:"""


### 2. Formula Fetcher

In [113]:
import json, re
from langchain.schema import HumanMessage       
from tenacity import retry, stop_after_attempt, wait_fixed

chat_formula = ChatOpenAI(
    model_name      = MODEL_NAME,
    temperature     = 0,
    max_tokens      = 256,
    openai_api_key  = None,
    # the API itself can insist on JSON if you have gpt‑4o/3.5‑turbo‑0125+:
    model_kwargs   = {"response_format": {"type": "json_object"}},
)

@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def _ask_formula(term: str, domain: str) -> dict:
    prompt = _GET_FORMULA_TMPL.format(term=term, domain=domain)
    raw    = chat_formula.invoke(prompt).content.strip()

    # 1️⃣ strip code‑fences if present
    if raw.startswith("```"):
        raw = raw.split("```")[1].strip()

    # 2️⃣ drop a leading language identifier such as “json”
    if raw.lower().startswith("json"):
        raw = raw[4:].lstrip()          # remove “json” + any following spaces/newlines

    # 3️⃣ (extra‑robust) grab just the JSON block
    first_brace = raw.find("{")
    last_brace  = raw.rfind("}")
    if first_brace == -1 or last_brace == -1:
        raise ValueError("No JSON object found in model response:\n" + raw)

    json_str = raw[first_brace : last_brace + 1]
    return json.loads(json_str)



def get_formula(jargon_rows: list[dict]) -> list[dict]:
    """
    Takes the *filtered* output from lc_relevance()
    (list of dicts with domain & jargon_term) and returns
    a **new list** that adds 'formula' and 'variables'.
    """
    enriched = []
    for row in jargon_rows:
        term   = row["jargon_term"]
        domain = row["domain"]

        try:
            data = _ask_formula(term, domain)
            row  = {**row, **data}           # merge dicts (PY≥3.9)
        except Exception as e:
            print(type(e).__name__, "→", e)          # <‑‑ add this
            row = {**row,
               "formula": None,
                "variables": {},
               "error": str(e)}
        
        enriched.append(row)

    return enriched


In [114]:
# relevant_rows = lc_relevance(pairs, cols, col_types, keep_only_formula=True)

formulas = get_formula(evaluated)

In [118]:
print(formulas)

[{'domain': 'Business Finance', 'jargon_term': 'EBITDA', 'relevant': True, 'has_formula': True, 'formula': 'Operating_Income + Depreciation + Amortization', 'variables': {'Operating_Income': 'Earnings from operations before interest & taxes', 'Depreciation': 'Expense for tangible asset depreciation', 'Amortization': 'Expense for intangible asset amortization'}}, {'domain': 'Business Finance', 'jargon_term': 'ROI', 'relevant': True, 'has_formula': True, 'formula': 'Net_Profit / Investment_Cost', 'variables': {'Net_Profit': 'Total profit generated by the investment', 'Investment_Cost': 'Initial cash outlay'}}, {'domain': 'Business Finance', 'jargon_term': 'DCF', 'relevant': True, 'has_formula': True, 'formula': 'sum(CF_t / (1 + r)**t for t in range(1, n+1))', 'variables': {'CF_t': 'Cash flow at time t', 'r': 'Discount rate', 't': 'Time period', 'n': 'Total number of periods'}}, {'domain': 'Business Finance', 'jargon_term': 'IRR', 'relevant': True, 'has_formula': True, 'formula': 'npv = s

In [116]:
df = pd.DataFrame(formulas)
df.head()

,domain,jargon_term,relevant,has_formula,formula,variables
0,Business Finance,EBITDA,True,True,Operating_Income + Depreciation + Amortization,{'Operating_Income': 'Earnings from operations...
1,Business Finance,ROI,True,True,Net_Profit / Investment_Cost,{'Net_Profit': 'Total profit generated by the ...
2,Business Finance,DCF,True,True,"sum(CF_t / (1 + r)**t for t in range(1, n+1))","{'CF_t': 'Cash flow at time t', 'r': 'Discount..."
3,Business Finance,IRR,True,True,npv = sum((Cash_Flow_t / (1 + IRR)**t) for t i...,"{'npv': 'Net present value, set to zero to sol..."
4,Business Finance,NPV,True,True,sum(Cash_Flow_t / (1 + Discount_Rate)**t for t...,{'Cash_Flow_t': 'Net cash inflow during the pe...


### 3. Prepare for Matcher

In [136]:
def extract_formula_keys(items):
    """
    Parameters
    ----------
    items : list[dict]
        The list of dictionaries containing formula information.

    Returns
    -------
    list[str]
        All formulas, deduplicated in order of first appearance.
    """
    seen, out = set(), []
    for d in items:
        formula = d.get("formula")
        if formula and formula not in seen:
            seen.add(formula)
            out.append(formula)
    return out


In [138]:
matcher_variables = extract_formula_keys(formulas)
print(matcher_variables)

['Operating_Income + Depreciation + Amortization', 'Net_Profit / Investment_Cost', 'sum(CF_t / (1 + r)**t for t in range(1, n+1))', 'npv = sum((Cash_Flow_t / (1 + IRR)**t) for t in range(1, n+1))', 'sum(Cash_Flow_t / (1 + Discount_Rate)**t for t in range(1, n+1)) - Initial_Investment', '(%_Change_in_Quantity_Demanded) / (%_Change_in_Price)']


## Agent 3: Formula Matcher

In [130]:
import re
import json
import asyncio
from typing import List, Dict, Any, Optional

from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

# ────────────────────────────────────────────────────────────────
# 0.  Helper — shared async run for notebooks that may already
#     have a running event‑loop (e.g. Jupyter, VSCode)
# ----------------------------------------------------------------
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    # ok if the package is missing in pure‑script execution
    pass


def run_async(coro):
    """Safe `await` helper that works in both scripts & notebooks."""
    try:
        return asyncio.run(coro)
    except RuntimeError as err:
        if "already running" in str(err):
            return asyncio.get_event_loop().run_until_complete(coro)
        raise

# ────────────────────────────────────────────────────────────────
# 1.  Initialise the LLM 
# ----------------------------------------------------------------

llm_matcher = ChatOpenAI(
    model_name      = MODEL_NAME,
    temperature     = TEMPERATURE,
    max_tokens      = 256,
    openai_api_key  = None,      # `None` → pull from env var automatically
    # the API itself can insist on JSON if you have gpt‑4o/3.5‑turbo‑0125+:
    model_kwargs   = {"response_format": {"type": "json_object"}}
)

# ────────────────────────────────────────────────────────────────
# 2.  Wrap the two external tools as tiny LLM chains
# ----------------------------------------------------------------
FUZZY_PROMPT = PromptTemplate(
    input_variables=["var", "columns"],
    template=(
        "You are a helper that aligns a short **variable** name to **one** of the "
        "candidate *column* names.\n\n"
        "Variable: {var}\n"
        "Candidates: {columns}\n\n"
        "Which candidate is the **best semantic match**?  Reply **JSON only**:\n"
        '{{"match":"<column_name>", "score": <0.0-1.0>}}'
    ),
)

fuzzy_chain = LLMChain(
    llm=llm_matcher,
    prompt=FUZZY_PROMPT,
    output_parser=StrOutputParser(),
)

DERIVE_PROMPT = PromptTemplate(
    input_variables=["var", "columns"],
    template=(
        "You are given a variable `{var}` and these table columns: {columns}.\n\n"
        "Can `{var}` be **computed** from the columns (via arithmetic or ratios)?\n"
        "If yes → output **exactly** the JSON:\n"
        '{{"expr":"<python-or-pandas-expression>"}} \n'
        "If impossible →  {{\"expr\": null}}"
    ),
)

derive_chain = LLMChain(
    llm=llm_matcher,
    prompt=DERIVE_PROMPT,
    output_parser=StrOutputParser(),
)

# ────────────────────────────────────────────────────────────────
# 3. ReAct‑style agent operating *per formula*
# ----------------------------------------------------------------
class FormulaAgent:
    """Single‑formula matcher that records every ReAct step."""

    _VAR_REGEX = re.compile(r"\b[A-Za-z_][A-Za-z0-9_]*\b")

    def __init__(
        self, formula: str, columns: List[str], *, threshold: float = 0.8
    ) -> None:
        self.formula = formula
        self.columns = columns
        self.threshold = threshold
        self.vars = self._extract_vars(formula)

        # ReAct trace & outputs
        self.log: List[Dict[str, Any]] = []
        self.mapping: Dict[str, str] = {}
        self.status: str = "pending"
        self.failure_reason: Optional[str] = None

    # ------------------------------------------------------------
    @staticmethod
    def _extract_vars(expr: str) -> List[str]:
        # crude: all identifiers that *start* with a letter/underscore
        return list(dict.fromkeys(FormulaAgent._VAR_REGEX.findall(expr)))

    # ------------------------------------------------------------
    async def _fuzzy(self, var: str) -> Dict[str, Any]:
        """Call fuzzy_match tool and parse JSON."""
        raw = await fuzzy_chain.arun(var=var, columns=self.columns)
        return json.loads(raw)

    async def _derive(self, var: str) -> Dict[str, Any]:
        raw = await derive_chain.arun(var=var, columns=self.columns)
        return json.loads(raw)

    # ------------------------------------------------------------
    async def run(self) -> Dict[str, Any]:
        for v in self.vars:
            # THOUGHT
            self.log.append({"step": "thought", "text": f"Match variable '{v}'"})

            # ACTION 1: fuzzy_match
            res = await self._fuzzy(v)
            self.log.append({"step": "action", "tool": "fuzzy_match", "input": v, "output": res})

            if res.get("score", 0) >= self.threshold:
                self.mapping[v] = res["match"]
                continue

            # THOUGHT again
            self.log.append({"step": "thought", "text": f"Try derive '{v}'"})

            # ACTION 2: can_be_derived
            der = await self._derive(v)
            self.log.append({"step": "action", "tool": "can_be_derived", "input": v, "output": der})

            if der.get("expr"):
                self.mapping[v] = der["expr"]
                continue

            # FAILURE — give up for this formula
            self.status = "fail"
            self.failure_reason = "derive_fail"
            break
        else:
            # completed loop without break ⇒ success!
            self.status = "saved"

        return {
            "formula": self.formula,
            "mapping": self.mapping,
            "status": self.status,
            "failure_reason": self.failure_reason,
            "log": self.log,
        }

# ────────────────────────────────────────────────────────────────
# 4.  Async batch helper
# ----------------------------------------------------------------
async def batch_match(formulas: List[str], columns: List[str]) -> List[Dict[str, Any]]:
    agents = [FormulaAgent(f, columns) for f in formulas]
    return await asyncio.gather(*[a.run() for a in agents])


In [139]:
# ────────────────────────────────────────────────────────────────
# 5.  Quick interactive demo (only runs when executed as a script)
# ----------------------------------------------------------------
if __name__ == "__main__":
    demo_columns = [
        "Quarter",
        "number_Customers",
        "Total_Transactions",
        "Revenue",
        "Profit",
    ]

    demo_formulas = matcher_variables

    output = run_async(batch_match(demo_formulas, demo_columns))
    from pprint import pprint

    pprint(output, width=120, compact=True)



[{'failure_reason': 'derive_fail',
  'formula': 'Operating_Income + Depreciation + Amortization',
  'log': [{'step': 'thought', 'text': "Match variable 'Operating_Income'"},
          {'input': 'Operating_Income',
           'output': {'match': 'Profit', 'score': 0.8},
           'step': 'action',
           'tool': 'fuzzy_match'},
          {'step': 'thought', 'text': "Match variable 'Depreciation'"},
          {'input': 'Depreciation',
           'output': {'match': 'Profit', 'score': 0.3},
           'step': 'action',
           'tool': 'fuzzy_match'},
          {'step': 'thought', 'text': "Try derive 'Depreciation'"},
          {'input': 'Depreciation', 'output': {'expr': None}, 'step': 'action', 'tool': 'can_be_derived'}],
  'mapping': {'Operating_Income': 'Profit'},
  'status': 'fail'},
 {'failure_reason': 'derive_fail',
  'formula': 'Net_Profit / Investment_Cost',
  'log': [{'step': 'thought', 'text': "Match variable 'Net_Profit'"},
          {'input': 'Net_Profit', 'output': {'